In [ ]:
import pandas as pd
import statistics as s

In [ ]:
GENETIC_DIR='meta_analysis_mixed'
%env GENDIR={GENETIC_DIR}

In [ ]:
# copying genetc data extracted with dsub_subset_and_map_bgen_parallel.ipynb
!gsutil -m cp -R gs~/data/cov-ldsc/output/${GENDIR} .

In [ ]:
%%bash

# generating file with file names

file="./${GENDIR}/mergelist.txt"

if [ -f "$file" ] ; then
    rm "$file"
fi

for i in {1..22}
do
echo "chr"${i}"_map" >> ./${GENDIR}/mergelist.txt
done

In [ ]:
%%bash

#merging per chromosome bed file into a single bed/bim/fam
cd ./${GENDIR}
plink2 --pmerge-list mergelist.txt bfile --make-bed --out merged_map

In [ ]:
%%bash

# LD pruning
cd ./${GENDIR}
plink2 --bfile merged_map --indep-pairwise 500 50 0.1

In [ ]:
%%bash

# estimating relatedness
cd ./${GENDIR}
plink2 --bfile merged_map --extract plink2.prune.in --king-cutoff 0.0884

In [ ]:
%%bash

# calculating PCA
cd ./${GENDIR}
plink2 --bfile merged_map --keep plink2.king.cutoff.in.id --extract plink2.prune.in  --pca

In [ ]:
# saving PCs as input file for cov-LDSC removing header
df = pd.read_csv("./" + GENETIC_DIR + "/plink2.eigenvec", sep = '\t')
df.to_csv('gs~/data/cov-ldsc/ldsc_input/' + GENETIC_DIR + '_plink2.eigenvec', 
          sep='\t', 
          index=False,
          header = False)

In [ ]:
%%bash

# removing related individuals from test chromosome
cd ./${GENDIR}

for i in {1..22};
do
    plink2 --bfile chr${i}_map --keep plink2.king.cutoff.in.id --make-bed --out chr${i}_map_unrelated
done

In [ ]:
# copying plink files containing only unrelated individuals to cloud
!gsutil -m cp ./${GENDIR}/chr*_map_unrelated* gs~/data/cov-ldsc/ldsc_input/${GENDIR}/